# hoodie.datasource.read.incr.fallback.fulltablescan.enable Settings Explained



![Untitled Diagram drawio-2](https://github.com/microsoft/botframework-sdk/assets/39345855/e1407871-bb8b-43b6-9057-4e60ec1319cf)


In [1]:
try:
    import os
    import sys
    import uuid
    import pyspark
    import datetime
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    from faker import Faker
    import datetime
    from datetime import datetime
    import random 
    import pandas as pd  # Import Pandas library for pretty printing

    print("Imports loaded ")

except Exception as e:
    print("error", e)

Imports loaded 


In [38]:
HUDI_VERSION = '1.0.0-beta1'
SPARK_VERSION = '3.4'

os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"
SUBMIT_ARGS = f"--packages org.apache.hudi:hudi-spark{SPARK_VERSION}-bundle_2.12:{HUDI_VERSION} pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
os.environ['PYSPARK_PYTHON'] = sys.executable

# Spark session
spark = SparkSession.builder \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.sql.extensions', 'org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
    .config('className', 'org.apache.hudi') \
    .config('spark.sql.hive.convertMetastoreParquet', 'false') \
    .getOrCreate()

In [47]:
def write_to_hudi(spark_df,
                  table_name,
                  db_name,
                  method='upsert',
                  table_type='COPY_ON_WRITE',
                  recordkey='',
                  precombine='',
                  partition_fields='',
                  index_type='BLOOM',
                  curr_region='us-east-1'
                  ):
    path = f"file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database={db_name}/table_name={table_name}"

    hudi_options = {
        'hoodie.table.name': table_name,
        'hoodie.datasource.write.table.type': table_type,
        'hoodie.datasource.write.table.name': table_name,
        'hoodie.datasource.write.operation': method,
        'hoodie.datasource.write.recordkey.field': recordkey,
        'hoodie.datasource.write.precombine.field': precombine,
        "hoodie.datasource.write.partitionpath.field": partition_fields,
        
        "hoodie.keep.min.commits":"5",
        "hoodie.cleaner.commits.retained":"5",
        "hoodie.keep.max.commits":"6",
       
        

        "hoodie.write.concurrency.mode": "optimistic_concurrency_control",
        "hoodie.cleaner.policy.failed.writes": "LAZY",
        "hoodie.write.lock.provider": "org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider",
    }
    print(hudi_options)

    print("\n")
    print(path)
    print("\n")

    spark_df.write.format("hudi"). \
        options(**hudi_options). \
        mode("append"). \
        save(path)

# Create Spark Df

In [48]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

    
schema = StructType([
    StructField("id", StringType(), True),
    StructField("message", StringType(), True)
])


# Loop to generate data and write to Hudi
for i in range(1, 10):  # Number of iterations
    # Generate epoch timestamp
    epoch_time = int(datetime.now().timestamp())

    # Create the data
    updated_data = [(str(i), "Batch : {} ".format(i))]

    # Create the DataFrame with the new data
    df = spark.createDataFrame(updated_data, schema)

    # Show the DataFrame with the updated "message" column
    df.show()

    # Write to Hudi
    write_to_hudi(
        spark_df=df,
        method="upsert",
        db_name="default",
        table_name="messages",
        recordkey="id",
        precombine="message"
    )

    import time
    time.sleep(1)


+---+----------+
| id|   message|
+---+----------+
|  1|Batch : 1 |
+---+----------+

{'hoodie.table.name': 'messages', 'hoodie.datasource.write.table.type': 'COPY_ON_WRITE', 'hoodie.datasource.write.table.name': 'messages', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.recordkey.field': 'id', 'hoodie.datasource.write.precombine.field': 'message', 'hoodie.datasource.write.partitionpath.field': '', 'hoodie.keep.min.commits': '5', 'hoodie.keep.max.commits': '6', 'hoodie.cleaner.commits.retained': '5', 'hoodie.write.concurrency.mode': 'optimistic_concurrency_control', 'hoodie.cleaner.policy.failed.writes': 'LAZY', 'hoodie.write.lock.provider': 'org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider'}


file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages




24/06/10 17:10:26 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=5 to be greater than hoodie.cleaner.commits.retained=5 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:26 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:29 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.


+---+----------+
| id|   message|
+---+----------+
|  2|Batch : 2 |
+---+----------+

{'hoodie.table.name': 'messages', 'hoodie.datasource.write.table.type': 'COPY_ON_WRITE', 'hoodie.datasource.write.table.name': 'messages', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.recordkey.field': 'id', 'hoodie.datasource.write.precombine.field': 'message', 'hoodie.datasource.write.partitionpath.field': '', 'hoodie.keep.min.commits': '5', 'hoodie.keep.max.commits': '6', 'hoodie.cleaner.commits.retained': '5', 'hoodie.write.concurrency.mode': 'optimistic_concurrency_control', 'hoodie.cleaner.policy.failed.writes': 'LAZY', 'hoodie.write.lock.provider': 'org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider'}


file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages




24/06/10 17:10:31 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=5 to be greater than hoodie.cleaner.commits.retained=5 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:32 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:33 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.


+---+----------+
| id|   message|
+---+----------+
|  3|Batch : 3 |
+---+----------+

{'hoodie.table.name': 'messages', 'hoodie.datasource.write.table.type': 'COPY_ON_WRITE', 'hoodie.datasource.write.table.name': 'messages', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.recordkey.field': 'id', 'hoodie.datasource.write.precombine.field': 'message', 'hoodie.datasource.write.partitionpath.field': '', 'hoodie.keep.min.commits': '5', 'hoodie.keep.max.commits': '6', 'hoodie.cleaner.commits.retained': '5', 'hoodie.write.concurrency.mode': 'optimistic_concurrency_control', 'hoodie.cleaner.policy.failed.writes': 'LAZY', 'hoodie.write.lock.provider': 'org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider'}


file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages




...

24/06/10 17:10:53 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=5 to be greater than hoodie.cleaner.commits.retained=5 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:53 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:54 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:56 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 

+---+----------+
| id|   message|
+---+----------+
|  8|Batch : 8 |
+---+----------+

{'hoodie.table.name': 'messages', 'hoodie.datasource.write.table.type': 'COPY_ON_WRITE', 'hoodie.datasource.write.table.name': 'messages', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.recordkey.field': 'id', 'hoodie.datasource.write.precombine.field': 'message', 'hoodie.datasource.write.partitionpath.field': '', 'hoodie.keep.min.commits': '5', 'hoodie.keep.max.commits': '6', 'hoodie.cleaner.commits.retained': '5', 'hoodie.write.concurrency.mode': 'optimistic_concurrency_control', 'hoodie.cleaner.policy.failed.writes': 'LAZY', 'hoodie.write.lock.provider': 'org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider'}


file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages




24/06/10 17:10:59 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=5 to be greater than hoodie.cleaner.commits.retained=5 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:10:59 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:11:00 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:11:02 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 

+---+----------+
| id|   message|
+---+----------+
|  9|Batch : 9 |
+---+----------+

{'hoodie.table.name': 'messages', 'hoodie.datasource.write.table.type': 'COPY_ON_WRITE', 'hoodie.datasource.write.table.name': 'messages', 'hoodie.datasource.write.operation': 'upsert', 'hoodie.datasource.write.recordkey.field': 'id', 'hoodie.datasource.write.precombine.field': 'message', 'hoodie.datasource.write.partitionpath.field': '', 'hoodie.keep.min.commits': '5', 'hoodie.keep.max.commits': '6', 'hoodie.cleaner.commits.retained': '5', 'hoodie.write.concurrency.mode': 'optimistic_concurrency_control', 'hoodie.cleaner.policy.failed.writes': 'LAZY', 'hoodie.write.lock.provider': 'org.apache.hudi.client.transaction.lock.FileSystemBasedLockProvider'}


file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages




24/06/10 17:11:05 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=5 to be greater than hoodie.cleaner.commits.retained=5 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:11:05 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:11:06 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 to be greater than hoodie.cleaner.commits.retained=20 (there is risk of incremental pull missing data from few instants based on the current configuration). The Hudi archiver will automatically adjust the configuration regardless.
24/06/10 17:11:07 WARN HoodieWriteConfig: Increase hoodie.keep.min.commits=6 

# Read From Hudi 

In [56]:
path = "file:///Users/soumilshah/IdeaProjects/SparkProject/tem/database=default/table_name=messages"


spark.read.format("hudi") \
    .load(path) \
    .createOrReplaceTempView("hudi_snapshot1")

query = f"SELECT _hoodie_commit_time, id, message FROM hudi_snapshot1 "
print(query)
result = spark.sql(query)
result.show(n=result.count(), truncate=False)


SELECT _hoodie_commit_time, id, message FROM hudi_snapshot1 
+-------------------+---+----------+
|_hoodie_commit_time|id |message   |
+-------------------+---+----------+
|20240610171026457  |1  |Batch : 1 |
|20240610171032011  |2  |Batch : 2 |
|20240610171036210  |3  |Batch : 3 |
|20240610171040032  |4  |Batch : 4 |
|20240610171044289  |5  |Batch : 5 |
|20240610171048431  |6  |Batch : 6 |
|20240610171053361  |7  |Batch : 7 |
|20240610171059290  |8  |Batch : 8 |
|20240610171105111  |9  |Batch : 9 |
+-------------------+---+----------+



# Incremental Query 

In [57]:
beginTime = "20240610171026456"

incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
  "hoodie.datasource.read.incr.fallback.fulltablescan.enable":"true"
}

IncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(path)

IncrementalDF.createOrReplaceTempView("hudi_incremental")
query = f"SELECT _hoodie_commit_time, id, message FROM hudi_incremental "

result = spark.sql(query)
result.show(n=result.count(), truncate=False)


+-------------------+---+----------+
|_hoodie_commit_time|id |message   |
+-------------------+---+----------+
|20240610171026457  |1  |Batch : 1 |
|20240610171032011  |2  |Batch : 2 |
|20240610171036210  |3  |Batch : 3 |
|20240610171040032  |4  |Batch : 4 |
|20240610171044289  |5  |Batch : 5 |
|20240610171048431  |6  |Batch : 6 |
|20240610171053361  |7  |Batch : 7 |
|20240610171059290  |8  |Batch : 8 |
|20240610171105111  |9  |Batch : 9 |
+-------------------+---+----------+

